In [2]:
# IMPORTS
from nltk.corpus import stopwords
import nltk
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np
import gc
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix



In [3]:
df = pd.read_csv('IMDB Dataset.csv')
df.head(5)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(50000, 2)

In [0]:
# #several data analysis first?
# import plotly.express as px
# fig = px.histogram(df,x='sentiment')
# fig.show()
# #positive case equals to negative case

# print(len(df['review'].unique()))
# #unique review:49582

# df = df.drop_duplicates()
# print(len(df))
# #we drop the duplicate and the remain number of the whole dataset is 49582.

# #some pre-process
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df['review'],df['sentiment'],test_size = 0.2)

# print(y_train.value_counts())
# print(y_test.value_counts())

In [0]:
# #Define function for removing special characters
# def RemoveNoisyCharacters(text):
#     soup = BeautifulSoup(text, "html.parser") #Remove HTML tags like <br>...
#     text = soup.get_text()
#     pattern=r'[^a-zA-z0-9\s]'                #Remove special characters
#     text = re.sub(pattern,'',text)
#     text = re.sub('\[[^]]*\]', '', text)     #Remove square brackets
#     return text
# #Apply function on review column
# df['review']=df['review'].apply(RemoveNoisyCharacters)

In [0]:
# #Remove Stop Words
# def removeStopWords(wordList):
#     newWordList = []
#     for word in wordList:
#         # Remove Stop Words
#         word = word.lower()
#         if word.lower() in stopwords:
#             continue
#         else:
#             newWordList.append(word)
#     return newWordList
# df['review']=df['review'].apply(RemoveNoisyCharacters)


In [4]:
#### Now Sun steps into the battlefield.
nltk.download('stopwords')
stops = set(stopwords.words('english'))
preprocess_df = pd.DataFrame(columns=['review', 'sentiment'], index=[i for i in range(len(df))])
for i in range(len(df)):
    sentence = df.loc[i,'review']
    text = BeautifulSoup(sentence, "html.parser").get_text()
    pattern=r'[^a-zA-z0-9\s]'                #Remove special characters
    text = re.sub(pattern,'',text)
    text = re.sub('\[[^]]*\]', '', text)     #Remove square brackets
    wordList = text.split()
    newWordList = []
    for word in wordList:
        # Remove Stop Words
        word = word.lower()
        if word.lower() in stops:
            continue
        else:
            newWordList.append(word)
    text = ' '.join(newWordList)
    preprocess_df.loc[i,'review'] = text
    if df.loc[i,'sentiment'] == 'positive':
        preprocess_df.loc[i,'sentiment'] = 1
        df.loc[i,'sentiment'] = 1
    else:
        preprocess_df.loc[i,'sentiment'] = -1
        df.loc[i,'sentiment'] = -1
print(preprocess_df.head(5))
preprocess_df = preprocess_df.drop_duplicates()
preprocess_df = preprocess_df.reset_index(drop = True)
#we drop the duplicate and the remain number of the whole dataset is 49582.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(preprocess_df['review'],preprocess_df['sentiment'],test_size = 0.2)

print(y_train.value_counts())
print(y_test.value_counts())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/shared-libs/python3.9/py-core/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
                                              review sentiment
0  one reviewers mentioned watching 1 oz episode ...         1
1  wonderful little production filming technique ...         1
2  thought wonderful way spend time hot summer we...         1
3  basically theres family little boy jake thinks...        -1
4  petter matteis love time money visually stunni...         1
 1    19835
-1    19828
Name: sentiment, dtype: int64
 1    5048
-1    4868
Name: sentiment, dtype: int64


In [ ]:
import math
sentence_idf = dict()
for i in range(len(X_train)):
    wordlist = X_train.iloc[i].split()
    wordset = set(wordlist)
    for word in wordset:
        if word in sentence_idf:
            sentence_idf[word] += 1
        else:
            sentence_idf[word] = 1
for word, value in sentence_idf.items():
    sentence_idf[word] = math.log(len(X_train)/value)

print(sentence_idf["love"])
print(sentence_idf["worst"])

1.7390905236458678
2.431950718983582


In [ ]:
def get_tfidf(review, sentenceidf = sentence_idf):
    wordlist = review.split()
    length = len(wordlist)
    setwordlist = set(wordlist)
    sentence_tf = dict.fromkeys(setwordlist, 0)
    for word in wordlist:
        sentence_tf[word] += 1/length
    sentence_tfidf = dict()
    for word, value in sentence_tf.items():
        if word in sentence_idf:
            sentence_tfidf[word] = value * sentence_idf[word]
    return sentence_tfidf

def predition_through_tfidf(review,X_train = X_train,y_train = y_train, sentenceidf = sentence_idf):
    sentence_tfidf = get_tfidf(review)
    vote = 0
    for i in range(len(X_train)):
        text = X_train.iloc[i]
        X_train_tfidf = get_tfidf(text, sentenceidf)
        for word, value in sentence_tfidf.items():
            if word in X_train_tfidf:
                vote += sentence_tfidf[word]*X_train_tfidf[word]*y_train.iloc[i]
    return vote

from sklearn.metrics import classification_report
from tqdm import tqdm
y_pred_real_num = []
y_pred = []
for i in tqdm(range(100)): 
#3.5s for one prediction, we can only predict first 100 to cut down the time cost
    num = predition_through_tfidf(X_test.iloc[i])
    y_pred_real_num.append(num)
    if num > 0:
        y_pred.append(1)
    else:
        y_pred.append(-1)

In [ ]:
y_true = []
for i in range(100):
    y_true.append(y_test.iloc[i])
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

          -1       0.82      0.78      0.80        51
           1       0.78      0.82      0.80        49

    accuracy                           0.80       100
   macro avg       0.80      0.80      0.80       100
weighted avg       0.80      0.80      0.80       100



In [ ]:
#what if we directly use the method without preprocessing?
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['review'],df['sentiment'],test_size = 0.2)
import math
sentence_idf = dict()
for i in range(len(X_train)):
    wordlist = X_train.iloc[i].split()
    wordset = set(wordlist)
    for word in wordset:
        if word in sentence_idf:
            sentence_idf[word] += 1
        else:
            sentence_idf[word] = 1
for word, value in sentence_idf.items():
    sentence_idf[word] = math.log(len(X_train)/value)
from sklearn.metrics import classification_report
from tqdm import tqdm
y_pred_real_num = []
y_pred = []
for i in tqdm(range(100)): 
#3.5s for one prediction, we can only predict first 100 to cut down the time cost
    num = predition_through_tfidf(X_test.iloc[i],X_train,y_train,sentence_idf)
    y_pred_real_num.append(num)
    if num > 0:
        y_pred.append(1)
    else:
        y_pred.append(-1)
y_true = []
for i in range(100):
    y_true.append(y_test.iloc[i])

print(classification_report(y_true,y_pred))
#We can see that without preprocess, the accuracy actually improves which implies that "stopwords" 
#and other things that we think it is unimportant actually can contribute positively to the performance
#However, without preprocess the model requires 12s instead of 3s for 1 prediction so it is still wise to preprocess 

100%|██████████| 100/100 [20:24<00:00, 12.24s/it]              precision    recall  f1-score   support

          -1       0.86      0.93      0.89        55
           1       0.90      0.82      0.86        45

    accuracy                           0.88       100
   macro avg       0.88      0.87      0.88       100
weighted avg       0.88      0.88      0.88       100




In [ ]:
#now I start to do the word2vec
!pip install gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
cf = df['review'].apply(simple_preprocess)
model = Word2Vec()
model.build_vocab(cf)
model.train(cf,total_examples=model.corpus_count, epochs=model.epochs)
model.save('./word2vec.model')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 34.9 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
cf

0        [one, of, the, other, reviewers, has, mentione...
1        [wonderful, little, production, br, br, the, f...
2        [thought, this, was, wonderful, way, to, spend...
3        [basically, there, family, where, little, boy,...
4        [petter, mattei, love, in, the, time, of, mone...
                               ...                        
49995    [thought, this, movie, did, down, right, good,...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [am, catholic, taught, in, parochial, elementa...
49998    [going, to, have, to, disagree, with, the, pre...
49999    [no, one, expects, the, star, trek, movies, to...
Name: review, Length: 50000, dtype: object

In [ ]:
!pip install gensim
from gensim.models import Word2Vec
import numpy as np
model = Word2Vec.load('word2vec.model')
voc = model.wv
X_train, X_test, y_train, y_test = train_test_split(preprocess_df['review'],preprocess_df['sentiment'],test_size = 0.2)
def average_vector(sentence, voc, vector_length = 100):
    wordlist = sentence.split()
    length = len(wordlist)
    avg_vector = np.zeros((vector_length,), dtype = float)
    count = 0
    for i in range(length):
        word = wordlist[i]
        if word in voc:
            avg_vector += voc[word]
            count += 1
    if count != 0:
        avg_vector/= count
    else:
        avg_vector = np.zeros((vector_length,), dtype = float)
    return avg_vector

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
vector_train = []
vector_test = []
for i in range(len(X_train)):
    vector_train.append(average_vector(X_train.iloc[i],voc))
for i in range(len(X_test)):
    vector_test.append(average_vector(X_test.iloc[i],voc))
print("start train:")
forest = forest.fit(vector_train,y_train.astype(int))
print("start test:")
y_pred = forest.predict(vector_test)
from sklearn.metrics import classification_report
print(classification_report(y_test.astype(int),y_pred))

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
start train:
start test:
              precision    recall  f1-score   support

          -1       0.83      0.80      0.82      4938
           1       0.81      0.84      0.82      4978

    accuracy                           0.82      9916
   macro avg       0.82      0.82      0.82      9916
weighted avg       0.82      0.82      0.82      9916



In [ ]:
#maybe average is not a good idea what about we add the weight of the tf idf value?
import math
sentence_idf = dict()
for i in range(len(X_train)):
    wordlist = X_train.iloc[i].split()
    wordset = set(wordlist)
    for word in wordset:
        if word in sentence_idf:
            sentence_idf[word] += 1
        else:
            sentence_idf[word] = 1
for word, value in sentence_idf.items():
    sentence_idf[word] = math.log(len(X_train)/value)

def get_tfidf(review, sentenceidf = sentence_idf):
    wordlist = review.split()
    length = len(wordlist)
    setwordlist = set(wordlist)
    sentence_tf = dict.fromkeys(setwordlist, 0)
    for word in wordlist:
        sentence_tf[word] += 1/length
    sentence_tfidf = dict()
    for word, value in sentence_tf.items():
        if word in sentence_idf:
            sentence_tfidf[word] = value * sentence_idf[word]
    return sentence_tfidf

def weighted_average_vector(sentence, voc, vector_length = 100, sentenceidf = sentence_idf):
    wordlist = sentence.split()
    sentence_tfidf = get_tfidf(sentence,sentenceidf)
    length = len(wordlist)
    avg_vector = np.zeros((vector_length,), dtype = float)
    count = 0
    for i in range(length):
        word = wordlist[i]
        if word in voc and word in sentence_tfidf:
            # avg_vector += voc[word]*sentenceidf[word]
            # count += sentenceidf[word]
            avg_vector += voc[word]*sentence_tfidf[word]
            count += sentence_tfidf[word]
    if count != 0:
        avg_vector/= count
    else:
        avg_vector = np.zeros((vector_length,), dtype = float)
    return avg_vector


model = Word2Vec.load('word2vec.model')
voc = model.wv
forest = RandomForestClassifier()
vector_train = []
vector_test = []
from tqdm import tqdm
for i in tqdm(range(len(X_train))):
    vector_train.append(weighted_average_vector(X_train.iloc[i],voc))
for i in tqdm(range(len(X_test))):
    vector_test.append(weighted_average_vector(X_test.iloc[i],voc))
print("start train:")
forest = forest.fit(vector_train,y_train.astype(int))
print("start test:")
y_pred = forest.predict(vector_test)
from sklearn.metrics import classification_report
print(classification_report(y_test.astype(int),y_pred))
#We can see that the tfidf weight is not working, it does not improve the result

100%|██████████| 9916/9916 [00:05<00:00, 1695.33it/s]
start train:
start test:
              precision    recall  f1-score   support

          -1       0.82      0.78      0.80      4938
           1       0.79      0.83      0.81      4978

    accuracy                           0.80      9916
   macro avg       0.80      0.80      0.80      9916
weighted avg       0.80      0.80      0.80      9916



In [1]:
#so maybe just to take an average is not enough for the text, we need to use the information of the context.
#As a result we may need to use some kind of RNN.
# !pip install tensorflow
# !pip install gensim
from tensorflow import keras
from keras.initializers import Constant
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
X_train, X_test, y_train, y_test = train_test_split(preprocess_df['review'],preprocess_df['sentiment'],test_size = 0.2)
X_train = X_train.reset_index()
X_test = X_test.reset_index()
y_train = y_train.reset_index()
y_test = y_test.reset_index()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocess_df['review'])
sequence_train = tokenizer.texts_to_sequences(X_train)
sequence_test = tokenizer.texts_to_sequences(X_test)
x_train = keras.preprocessing.sequence.pad_sequences(sequence_train,padding = 'post',maxlen = 100)
x_test = keras.preprocessing.sequence.pad_sequences(sequence_test,padding = 'post',maxlen = 100)
vocab_size = len(tokenizer.word_index) + 1

from tqdm import tqdm
word2vec_model = Word2Vec.load('word2vec.model')
voc = word2vec_model.wv
embedding_matrix = dict()
embedding_matrix[0] = np.zeros((100,), dtype = float)
for word in tokenizer.word_index:
    idx = tokenizer.word_index[word]
    if word in voc:
        embedding_matrix[idx] = voc[word]
    else:
        embedding_matrix[idx] = np.zeros((100,), dtype = float)

LSTM_model = keras.Sequential()
LSTM_model.add(keras.layers.Embedding(input_dim=vocab_size,output_dim=100,embeddings_initializer=Constant(embedding_matrix)))
LSTM_model.add(keras.layers.LSTM(64,return_sequences = True))
LSTM_model.add(keras.layers.LSTM(64,return_sequences = True))
LSTM_model.add(keras.layers.LSTM(128,return_sequences = True))
LSTM_model.add(keras.layers.Dense(10,activation='relu'))
LSTM_model.add(keras.layers.Dense(1,activation='sigmoid'))
opt = keras.optimizers.Adam(lr = 0.001)
LSTM_model.compile(optimizer = opt, loss = 'binary_crossentropy',metrics=['acc'])
LSTM_model.fit(x_train,y_train.astype(int),epochs=20,validation_data=(X_test,y_test.astype(int)),batch_size = 64)
loss, acc = LSTM_model.evaluate(x_test, y_test, verbose = False)
print("Testing accuracy: ",loss, acc)


In [ ]:
tokenizer.word_index

{'movie': 1,
 'film': 2,
 'one': 3,
 'like': 4,
 'good': 5,
 'even': 6,
 'would': 7,
 'time': 8,
 'really': 9,
 'see': 10,
 'story': 11,
 'much': 12,
 'well': 13,
 'get': 14,
 'great': 15,
 'bad': 16,
 'also': 17,
 'people': 18,
 'first': 19,
 'dont': 20,
 'movies': 21,
 'made': 22,
 'films': 23,
 'make': 24,
 'could': 25,
 'way': 26,
 'characters': 27,
 'think': 28,
 'watch': 29,
 'many': 30,
 'seen': 31,
 'character': 32,
 'two': 33,
 'never': 34,
 'love': 35,
 'acting': 36,
 'best': 37,
 'plot': 38,
 'little': 39,
 'know': 40,
 'show': 41,
 'life': 42,
 'ever': 43,
 'better': 44,
 'say': 45,
 'still': 46,
 'scene': 47,
 'end': 48,
 'man': 49,
 'scenes': 50,
 'something': 51,
 'go': 52,
 'back': 53,
 'real': 54,
 'im': 55,
 'watching': 56,
 'thing': 57,
 'doesnt': 58,
 'didnt': 59,
 'actors': 60,
 'years': 61,
 'actually': 62,
 'though': 63,
 'makes': 64,
 'funny': 65,
 'another': 66,
 'find': 67,
 'nothing': 68,
 'look': 69,
 'going': 70,
 'work': 71,
 'lot': 72,
 'new': 73,
 'every

In [0]:
# #SKLEARN FRAMEWORK IMPLEMENTATION

# #Count vectorizer for bag of words
# cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
# #transformed train reviews
# cv_reviews=cv.fit_transform(df['review'])
# #transformed test reviews

# #TF_IDF VECTORIZER
# tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
# #transformed train reviews
# tv_reviews=tv.fit_transform(df['review'])
# print(tv_reviews)

# #dataset split
# X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.2, random_state=42)

# #Multinomial Naive Bayes Model
# nb_classifier = MultinomialNB()
# nb_classifier.fit(X_train, y_train)

# # Make predictions on the testing set
# y_pred = nb_classifier.predict(X_test)

# nb_classifier_report = classification_report(y_test,y_pred,target_names=['Positive','Negative'])
# print(nb_classifier_report)
# confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
# cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
# cm_display.plot()
# plt.show()


In [0]:
# #TF-IDF Part Implementation Without SkLearn
# def calculateTermFrequency(wordList,numberOfDocument,documentIndex):
#     wordList = removeStopWords(wordList)
#     numberOfWords = len(wordList)
#     termFrequency = {}
#     for word in wordList:
#         if word not in termFrequency:
#             termFrequency[word] = 1/ numberOfWords
#         else:
#             termFrequency[word] = (1/numberOfWords) + termFrequency[word]
        
#         #calculateInverseDocumentFrequency
#         if word in inverseDocumentFrequency and termFrequency[word] == (1/numberOfWords): # if the word exist in previous documents but not in this document yet then update it 
#             documentFrequency[word] += 1
#             inverseDocumentFrequency[word] = np.log(numberOfDocument/documentFrequency[word])

#         elif word not in inverseDocumentFrequency:
#             documentFrequency[word] = 1
#             inverseDocumentFrequency[word] = np.log(numberOfDocument/1)
        
#         tf_idf[documentIndex, word] = termFrequency[word] * inverseDocumentFrequency[word]
        

#     return termFrequency




# #For the IDF part, each review is considered as a document. 
# #Apply function on review column
# def tfIDF():
#     inverseDocumentFrequency = {}
#     numberOfDocument = len(df['review'])
#     documentFrequency = {}
#     nltk.download('stopwords')
#     stopwords = nltk.corpus.stopwords.words('english') #STOP WORDS
#     tf_idf = {}

#     for index in df.index:
    
#         reviews = df['review'][index].lower()
#         wordList = reviews.split(' ')
#         termFrequencyForEachDocument = calculateTermFrequency(wordList,numberOfDocument,index)
#     return tf_idf

# tf_idf = tfIDF()


In [0]:
# del(stopwords)
# del(inverseDocumentFrequency)
# del(documentFrequency)

# gc.collect()

In [0]:

# sorted_tf_idf = sorted(tf_idf.items(), key=lambda x:x[1])
# len(sorted_tf_idf)

In [0]:
# def calculateTopThree():
#     topThreeList = pd.DataFrame(index=range(numberOfDocument))
#     for index in topThreeList.index:
#         first = {}
#         second = {}
#         third = {}

#         oneDocTfIdf = {}
#         for key, value in tf_idf.items():
#             if index == key[0]:
#                 oneDocTfIdf[key[0],key[1]] = (value) 
        
#         sorted_oneDocTfIdf = sorted(oneDocTfIdf.items(), key=lambda x:x[1])
#         first=sorted_oneDocTfIdf[0]
#         second=sorted_oneDocTfIdf[0]
#         third=sorted_oneDocTfIdf[0]

#This part took 35 minutes and gave this error
#KernelInterrupted: Execution interrupted by the Jupyter kernel.


In [0]:
# print(sorted_tf_idf[0]) #in row 31240, the word movie has lots of repetition. 
# print(sorted_tf_idf[1])
# print(sorted_tf_idf[2])
# print(sorted_tf_idf[4974040])

# print(np.log(numberOfDocument/1))


In [0]:
# highestTopThreeList = pd.DataFrame()
# for i,rows in enumarate(tf_idf):
#     if int(rows.keys()[0]) == i and 

OK so here is how we use the tf idf i think:

imagine we have a sentence "I love this movie." and we sort the highest tf-idf thing.  "love" 0.9, "I" 0.8 and form a sentence

And we select maybe 80% sentence as the trainning set and we use them to vote, for example, if 5000 cases in the trainning set with "love" says it is positive and 3000 cases with "love" says it is negative, after the vote we predict this is postive.

And each vote has certain weights. which means "love" has 0.9 votes and "I" has 0.8 votes.

negative or positive (prediction) I think i understand your point. Lets keep the notes!

For review in X_test:
          For word in review:
                  Dictionary(tf*idf) 
          Sort and store highest
          For word in dictionary:
                  For train_review in X_train:
                         Vote

TODO => Stemming

TODO =>Lemmatisation

TODO => Number Conversion

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1830d32f-c8ce-4fc8-8446-4b86a746869d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>